# Une fonction pour simplifier l'accès aux données

In [1]:
def extractField(s: String, fieldNumber: Int): String = {
    val fields = s.split('\t')
    if (fieldNumber >= fields.length) "" else fields(fieldNumber)
}

In [2]:
println(extractField("2	CASSIOPEE	2009	33	3.0000", 0))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 1))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 2))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 3))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 4))
println(extractField("2	CASSIOPEE	2009	33	3.0000", 5))

2
CASSIOPEE
2009
33
3.0000


# Charger les données
1. Créer le RDD `lignes` à partir du répertoire `prenoms_sample.txt`

In [2]:
val lignes = sc.textFile("prenoms_sample.txt")
lignes.take(10).foreach(println)

2	LISON	2010	72	4.0000
2	LIVIA	2005	25	3.0000
2	LIVIA	2014	63	7.0000
2	LOANE	2003	16	4.0000
2	LOLA	1996	09	3.0000
2	LOLA	2014	93	13.0000
2	LOLA	2015	11	20.0000
2	LOLA	2015	64	38.0000
2	LOLITA	2011	59	3.0000
2	LORETTE	2002	13	3.0000


# Transformer les lignes en prénoms
1. En appliquant la méthode `map`, créer le RDD `prenoms` à partir de `lignes`

In [3]:
val prenoms = lignes.map(l => (
    extractField(l, 0).charAt(0),
    extractField(l, 1),
    extractField(l, 2).toInt,
    extractField(l, 3).toInt,
    extractField(l, 4).toDouble.toInt
))
prenoms.take(10).foreach(println)

(2,LISON,2010,72,4)
(2,LIVIA,2005,25,3)
(2,LIVIA,2014,63,7)
(2,LOANE,2003,16,4)
(2,LOLA,1996,9,3)
(2,LOLA,2014,93,13)
(2,LOLA,2015,11,20)
(2,LOLA,2015,64,38)
(2,LOLITA,2011,59,3)
(2,LORETTE,2002,13,3)


# Interroger les données
La documentation des méthodes d'un RDD est disponible ([RDD](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.RDD), [PairRDDFunctions](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.rdd.PairRDDFunctions)).

1. Rappeler ce que sont les *transformations* et les *actions*
1. Donner, pour chaque prénom, son nombre d'occurences (`map` et `reduceByKey`)

In [57]:
val result = prenoms.map(l => ( l._2,l._5)).reduceByKey((counter,n) => counter + n)
result.take(10).foreach(println)

(BRICE,16)
(THIMEO,4)
(NADIR,9)
(ASSIA,9)
(NAD�GE,8)
(NICOLLE,9)
(SOUMAYA,6)
(AM�LIE,75)
(DANIELE,139)
(ANNE-SOPHIE,20)


1. Donner le nombre total de naissances avec un prénom féminin (`filter`, `map`, `reduce` ou `sum`)

In [85]:
prenoms.filter(l => (l._1 == '2')).map(l => l._5).reduce((sum,n)=>(sum+n)).toString()

36521

1. Donner l'effectif maximal et minimal par prénom (`map`, `aggregateByKey`)

In [99]:
val result = prenoms.map(p => (p._2,p._5)).aggregateByKey((Int.MaxValue,Int.MinValue))(
{case ((m,n),v1) => (m min v1, n max v1)},
{case((m,n),(m1,n1)) => (m min m1, n max n1)});
result.take(10).foreach(println)

(BRICE,(4,7))
(THIMEO,(4,4))
(NADIR,(3,6))
(ASSIA,(9,9))
(NAD�GE,(4,4))
(NICOLLE,(4,5))
(SOUMAYA,(3,3))
(AM�LIE,(4,41))
(DANIELE,(20,75))
(ANNE-SOPHIE,(5,15))


1. Sur le modèle des prénoms, charger les données des départements
1. Donner, pour chaque nom de département, le prénom le plus fréquent depuis l'année 2000

In [140]:
val lignesDept = sc.textFile("depts.txt").filter(l => l.startsWith("REGION") == false).filter(
    l => l.contains("2A") == false).filter(l => l.contains("2B") == false)
val depts = lignesDept.map(l => (
    extractField(l, 0).toInt,
    extractField(l, 1).toInt,
    extractField(l, 2).toInt,
    extractField(l, 3).toInt,
    extractField(l, 4),
    extractField(l, 5)
))
depts.take(5).foreach(println)

(84,1,1053,5,AIN,Ain)
(32,2,2408,5,AISNE,Aisne)
(84,3,3190,5,ALLIER,Allier)
(93,4,4070,4,ALPES-DE-HAUTE-PROVENCE,Alpes-de-Haute-Provence)
(93,5,5061,4,HAUTES-ALPES,Hautes-Alpes)


In [205]:
val deptsJoin = depts.map(l => (l._2,l._5))
val prenomsJoin = prenoms.filter(l => l._3 >= 2000).map(l => (l._4,(l._2,l._5)))
val joins = deptsJoin.join(prenomsJoin)
println("\tjoins")
joins.take(2).foreach(println)
val deptPreCount = joins.map(l => (l._2._1,(l._2._2._1,l._2._2._2)))
println("\t--------")
println("\t(dept,(prenom, count))")
result.take(2).foreach(println)
val result = preUtil.aggregateByKey(("",Int.MinValue))(
{case ((m,n),(v1,v2)) =>( if (n > v2) m else v1,n max v2)},
{case ((m,n),(v1,v2)) =>( if (n > v2) m else v1,n max v2)})
println("\t--------")
println("\tresult")

result.take(10).foreach(println)


	joins
(84,(VAUCLUSE,(MYL�NE,3)))
(84,(VAUCLUSE,(NORAH,3)))
	--------
	(dept,(prenom, count))
(JURA,(ANNA,5))
(PYRENEES-ORIENTALES,(LOUISE,13))
	--------
	result
(JURA,(ANNA,5))
(PYRENEES-ORIENTALES,(LOUISE,13))
(PAS-DE-CALAIS,(THA�S,12))
(PUY-DE-DOME,(ANGELINA,10))
(DROME,(MAXIME,23))
(MAINE-ET-LOIRE,(ALEXIS,80))
(TARN,(SACHA,18))
(HAUTE-MARNE,(LENNY,9))
(YONNE,(CL�MENT,10))
(CHARENTE,(LIAM,10))
